In [1]:
# http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#many-to-many
import sqlalchemy
sqlalchemy.__version__ 

'1.2.4'

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [3]:
from sqlalchemy import Column, Integer, String
from sqlalchemy import Table, ForeignKey
from sqlalchemy.orm import relationship

association_table = Table('association', Base.metadata,
    Column('left_id', Integer, ForeignKey('left.id')),
    Column('right_id', Integer, ForeignKey('right.id'))
)

class Parent(Base):
    __tablename__ = 'left'
    id = Column(Integer, primary_key=True)
    children = relationship(
        "Child",
        secondary=association_table,
        back_populates="parents")

class Child(Base):
    __tablename__ = 'right'
    id = Column(Integer, primary_key=True)
    parents = relationship(
        "Parent",
        secondary=association_table,
        back_populates="children")
    
Base.metadata.create_all(engine)

2018-03-08 00:17:39,166 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-08 00:17:39,168 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:17:39,169 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-08 00:17:39,170 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:17:39,173 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("association")
2018-03-08 00:17:39,173 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:17:39,175 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("left")
2018-03-08 00:17:39,175 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:17:39,176 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("right")
2018-03-08 00:17:39,177 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:17:39,180 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "left" (
	id INTEGER NOT NULL, 
	PRIMARY KEY (id)
)


2018-03-08 00:17:39,181 INFO sqlalchemy.engine.base.Engine ()


In [4]:
s = Session()

child = Child()
parent = Parent()

s.add(child)
s.add(parent)

s.commit()

print('=' * 64)

s = Session()

child = s.query(Child).order_by(Child.id.desc()).first()  # find the last one
parent = s.query(Parent).order_by(Parent.id.desc()).first()
print('#' * 32)
 
assert child.parents == []
assert parent.children == []
print('=' * 32)


# print(child.parents.__dir__())
child.parents.append(parent)
print('#' * 32)

s.commit()
print('=' * 64)

2018-03-08 00:17:39,227 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-08 00:17:39,229 INFO sqlalchemy.engine.base.Engine INSERT INTO "left" DEFAULT VALUES
2018-03-08 00:17:39,230 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:17:39,231 INFO sqlalchemy.engine.base.Engine INSERT INTO "right" DEFAULT VALUES
2018-03-08 00:17:39,231 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:17:39,233 INFO sqlalchemy.engine.base.Engine COMMIT
2018-03-08 00:17:39,235 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-08 00:17:39,236 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right" ORDER BY "right".id DESC
 LIMIT ? OFFSET ?
2018-03-08 00:17:39,237 INFO sqlalchemy.engine.base.Engine (1, 0)
2018-03-08 00:17:39,240 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id 
FROM "left" ORDER BY "left".id DESC
 LIMIT ? OFFSET ?
2018-03-08 00:17:39,241 INFO sqlalchemy.engine.base.Engine (1, 0)
################################
2018-03-08 00:17:3